#Faiss
Facebook AI Similarity Search (Faiss) is an open-source library developed by Meta (Facebook AI) that helps perform efficient similarity search and clustering of high-dimensional vectors.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
docs=text_splitter.split_documents(documents)
docs


[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [2]:
#embeddings=OllamaEmbeddings()
embeddings = OllamaEmbeddings(model="mistral")
db=FAISS.from_documents(docs,embeddings)
db

C:\Users\Mausomi Paul\AppData\Local\Temp\ipykernel_33140\4101169619.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="mistral")


In [3]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [4]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [5]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='aef9855a-0760-44b5-8ccb-c1f6009f9792', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(177414.38)),
 (Document(id='5902635c-bb1f-429e-99c6-dfee91eafe42', metadata={'source': 'spe

In [6]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[-3.604959011077881,
 3.193011999130249,
 5.980410099029541,
 2.8891451358795166,
 4.80374813079834,
 -2.521841526031494,
 6.9623003005981445,
 12.949563980102539,
 5.65200662612915,
 -7.012495994567871,
 -7.343698024749756,
 -3.196058511734009,
 -10.730260848999023,
 11.43239688873291,
 -2.9287869930267334,
 -3.5297601222991943,
 -4.222616195678711,
 0.7658200860023499,
 9.186103820800781,
 0.9956014752388,
 -1.833789587020874,
 4.32511568069458,
 -1.0593911409378052,
 0.2627353370189667,
 -0.1413722038269043,
 2.6969776153564453,
 -4.563601493835449,
 -6.829663276672363,
 -6.656284809112549,
 -3.6376988887786865,
 6.939056396484375,
 2.880507707595825,
 -5.071178913116455,
 -2.0243265628814697,
 3.49543833732605,
 -1.1315160989761353,
 0.5076125860214233,
 4.342719554901123,
 2.745412588119507,
 1.8773133754730225,
 -3.54888916015625,
 -9.723225593566895,
 4.784964561462402,
 -1.3430536985397339,
 -9.157939910888672,
 -1.2459065914154053,
 -3.1590754985809326,
 2.903303861618042,
 -1

In [ ]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

In [ ]:
### Saving And Loading
db.save_local("faiss_index")

In [ ]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [ ]:
docs